#  PCA and CCA

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))

import panel as pn
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from bokeh.resources import INLINE

import functions_bondjango as bd
import functions_kinematic as fk
import functions_misc as fm
import functions_data_handling as fd
import pandas as pd
import numpy as np
import h5py
from sklearn.decomposition import PCA
# define the name to be used for the saved figures
save_name = 'PCA'
line_width = 5

In [ ]:
# load the data
# get the data paths
try:
    data_path = snakemake.input[0]
except NameError:
    # define the search string
    search_string = 'result:succ, lighting:normal, rig:miniscope, =analysis_type:aggFullCA'
    # query the database for data to plot
    data_all = bd.query_database('analyzed_data', search_string)
    data_path = data_all[0]['analysis_path']
print(data_path)

# load the data
data = fd.aggregate_loader(data_path)

In [ ]:
hv.extension('plotly')
# get the calcium data for this experiment
labels = list(calcium.columns)
cells = [el for el in labels if 'cell' in el]
calcium_data = calcium.loc[:, cells].to_numpy()
# create the PCA object
pca = PCA(n_components=3)
# reduce the dimensionality of the neural activity over time
reduced_ca = pca.fit_transform(calcium_data)

# plot the trajectory
curve3d = hv.Path3D(reduced_ca)*hv.Scatter3D(np.reshape(reduced_ca[0,:],(1,-1)))
curve3d

In [ ]:
# allocate dictionaries for both of the projections
dict_1 = {}
dict_2 = {}


# get the dates for this mouse
mice = data.keys()

# for all the mice
for mouse in mice:
    # allocate memory for the maps
    map_list = []
    
    # get the dates for this mouse
    dates = data[mouse].keys()

    # for all the dates
    for day in dates:
        # get the table
        sub_data = data[mouse][day]
        # isolate the calcium data
        columns = sub_data.columns
        cells = [el for el in columns if 'cell' in el]
        calcium_data = sub_data[cells].to_numpy()
        
        # create the PCA object
        pca = PCA(n_components=3)
        # reduce the dimensionality of the neural activity over time
        reduced_ca = pca.fit_transform(calcium_data)
        
        # plot the 2 projections
        projection1 = hv.Curve((reduced_ca[:, 0], reduced_ca[:, 1]), 'PC1', 'PC2')
        projection2 = hv.Curve((reduced_ca[:, 0], reduced_ca[:, 2]), 'PC1', 'PC3')
        
        # store in the dictionary for the holomap
        dict_1[(mouse, day)] = projection1
        dict_2[(mouse, day)] = projection2

# create the holomaps
holo1 = hv.HoloMap(dict_1, kdims=['mouse', 'day'])
holo2 = hv.HoloMap(dict_2, kdims=['mouse', 'day'])

# create the layout
holo_image = pn.panel(holo1.opts(shared_axes=False)+
                      holo2.opts(shared_axes=False), center=True, 
                      widget_location='top', widgets={'cell': pn.widgets.DiscreteSlider})

holo_image  